In [1]:
import numpy as np
import pandas as pd
import pickle

In [40]:
df = pd.read_csv('1_min_labeled.csv')

f = open('A.txt', 'rb')
A = pickle.load(f)


In [41]:
len(df)

101475

In [42]:
def calc_features(vehicles, ego_id, v_mean):
    
    ego = vehicles[ego_id]
    if ego.Preceding in vehicles.keys():
        front = vehicles[ego.Preceding]
    else:
        front = None
    
    if len(vehicles) > 1:
        v_mean = (v_mean * len(vehicles) - ego.v_Vel) / (len(vehicles)-1)
    
    lane_width = 12
    
    f = np.zeros(10)
    f[0] = lane_width
    f[1] = ego.v_Acc # because the angle is small, we use a to approximate longitudinal acceleration
    f[2] = 0 # No deceleration light info in data
    f[3] = ego.lateral_acc
    f[4] = ego.v_Vel - v_mean
    f[5] = ego.v_Vel - front.v_Vel if front is not None else 0
    f[6] = -1 if min(ego.Local_X % lane_width, lane_width - (ego.Local_X % lane_width)) - (ego.v_Width/2) < 0 else 0
    f[7] = 0
    f[8] = ego.lateral_vel # calculated by Local_X derivative
    f[9] = (ego.Local_X % lane_width) - (lane_width / 2)
    
    return f



In [43]:
dT = 0.1
x = list()
y = list()
vehicles = dict()
show_up = set()
vel_sum = 0
correct_cnt = 0
p = dict()
f = open('random_forest_changing.txt', 'rb')
rfc = pickle.load(f)

In [44]:
last_time = 0
avg_lateral_v = dict()
for i, r in df.iterrows():
    
    if i == len(df)-1 or r.Global_Time != last_time:
        v_ids = list(vehicles.keys())
        for v_id in v_ids:
            if v_id not in show_up:
                vehicles.pop(v_id)
        show_up = set()
    
    v_id = r.Vehicle_ID
    
    if v_id not in vehicles.keys():
        p[v_id] = np.array([0.1, 0.1, 0.8])
        avg_lateral_v[v_id] = 0

    show_up.add(v_id)
    vehicles[v_id] = r.copy()
    f = calc_features(vehicles, v_id, r.mean_vel)
    f = np.zeros(10)
    
    avg_lateral_v[v_id] = avg_lateral_v[v_id] * 0.9 + f[8]
    d = int(avg_lateral_v[v_id] > 0)

# #         PyB = measure_prob(f, changing_lane_coef, accelerating_coef)

    PyB = np.zeros(3)
    turning = rfc.predict_proba(f[np.newaxis,:])[0]
    PyB[2] = turning[0]
    PyB[d] = turning[1]


    if f[6] == -1:
        B = d
        PyB[B] = 1
    else:

        p[v_id] = np.multiply(PyB , np.matmul(p[v_id], A));
        p[v_id] = PyB;
        p[v_id] = p[v_id] / np.sum(p[v_id])
        B = np.argmax(p[v_id])

#     correct_cnt += r.behavior == B
#         print('curr p = ', p[v_id])
#         print('pred = ', B,  'gt = ', r.behavior)

#     remove exited car after every moment


    if i % 10000 == 0:
        print(i,)
print('acc: ', correct_cnt / len(df))


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
acc:  0.0
